In [ ]:
#important libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## Load dataset.

In [ ]:
data = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')

In [ ]:
data.head()
#print(data.tail())


In [ ]:
#some random values from our data
data.sample(5)

In [ ]:
data.info()

In [ ]:
data.describe()

## Data Cleaning

In [ ]:
#check is there any NaN value.
data.isnull().sum()

In [ ]:
#let's check outlier now.
plt.figure(figsize=(20,8))
plt.boxplot(data)
plt.show()

outliner seen in column 5


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sns.distplot(data["Insulin"])

Here, remove top 1% outlier in Insulin attribute   

In [ ]:
q=data['Insulin'].quantile(.99)
data1=data[data['Insulin']<q]

In [ ]:
sns.distplot(data1["Insulin"])

In [ ]:
plt.boxplot(data['Insulin'])
plt.show()

# EDA

Firstly lets check that if our data is balanced or not

In [ ]:
 bl=data['Outcome'].value_counts()

In [ ]:
bl.plot(kind='bar',rot=360,color='red')

Our Data is clearly not balanced. We will balance this data using SMOTE technique after some visualizations.

SMOTE is an oversampling technique where the synthetic samples are generated for the minority class. This algorithm helps to overcome the overfitting problem posed by random oversampling. It focuses on the feature space to generate new instances with the help of interpolation between the positive instances that lie together.


In [ ]:
data1.plot(kind= 'kde' , subplots=True, layout=(3,3), sharex=False, sharey=False, figsize=(15,10))
plt.show()

It seems like most attributes are normally distributed.

In [ ]:
# Create a correlation matrix
corr_metrics =data1.corr()

corr_metrics.style.background_gradient()

In [ ]:
# Age vs BloodPressure with hue = Outcome
plt.figure(figsize=(12,8))
ax = sns.scatterplot(x="BloodPressure", y="Age", alpha=0.4,label="0", data=data[data['Outcome'] == 0])
sns.scatterplot(x="BloodPressure", y="Age", alpha=1, label="1",data=data[data['Outcome'] == 1], ax=ax)
plt.show()

first, Bloodpressure never be zero

reference 
#https://www.nhs.uk/common-health-questions/lifestyle/what-is-blood-pressure/
2
https://www.quora.com/What-will-happen-when-the-blood-pressure-goes-zero

In [ ]:
data2 = data1[data1['BloodPressure'] != 0]

In [ ]:
#crosscheck
sum(data2['BloodPressure']==0)

In [ ]:
# Splitting into features and value to be predicted
X = data2.drop(columns=['Outcome'])
y = data2['Outcome']



In [ ]:
fig, ax = plt.subplots(1,2 ,figsize = (10,5))

sns.barplot(x=['0', '1'], y =[sum(y == 0), sum(y == 1)], ax = ax[0])
ax[0].set_title("Before Oversampling")
ax[0].set_xlabel('Outcome')

#Using SMOTE to balance the Data
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state = 2) 
X, y = sm.fit_resample(X, y) 

sns.barplot(x=['0', '1'], y =[sum(y == 0), sum(y == 1)], ax = ax[1])
ax[1].set_title("After Oversampling")
ax[1].set_xlabel('Outcome')

plt.tight_layout()
plt.show()

Split data into train and test 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40)


## SVM(support vector machine)

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler


pipe1 = make_pipeline(MinMaxScaler(), SVC())
pipe2 = make_pipeline(Normalizer(), SVC())
pipe3 = make_pipeline(RobustScaler(), SVC())
pipe4 = make_pipeline(StandardScaler(), SVC())

In [ ]:
parameter = {'svc__C': [0.001,0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1, 2, 5, 10, 20, 50, 100,200],
              'svc__gamma': [0.001,0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1, 2, 5, 10, 20, 50, 100,200]} 

In [ ]:
# for minmaxscaler
grid = GridSearchCV(pipe1,parameter, cv=5)
grid.fit(X_train, y_train)
print("Best cross-validation accuracy:", grid.best_score_)
print("Test set score:", grid.score(X_test, y_test))
print("Test error rate:", 1-grid.score(X_test, y_test))
print("Best parameters:", grid.best_params_)

In [ ]:
#for Normalizer
grid = GridSearchCV(pipe2,parameter, cv=5)
grid.fit(X_train, y_train)
print("Best cross-validation accuracy:", grid.best_score_)
print("Test set score:", grid.score(X_test, y_test))
print("Test error rate:", 1-grid.score(X_test, y_test))
print("Best parameters:", grid.best_params_)

In [ ]:
#RobustScaler
grid = GridSearchCV(pipe3,parameter, cv=5)
grid.fit(X_train, y_train)
print("Best cross-validation accuracy:", grid.best_score_)
print("Test set score:", grid.score(X_test, y_test))
print("Test error rate:", 1-grid.score(X_test, y_test))
print("Best parameters:", grid.best_params_)

In [ ]:
# StandardScaler
grid = GridSearchCV(pipe4,parameter, cv=5)
grid.fit(X_train, y_train)
print("Best cross-validation accuracy:", grid.best_score_)
print("Test set score:", grid.score(X_test, y_test))
print("Test error rate:", 1-grid.score(X_test, y_test))
print("Best parameters:", grid.best_params_)

In [ ]:
prediction = grid.predict(X_test)
prediction

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,prediction)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,prediction)